# Ejercicio 1: Introducción a Recuperación de Información

## Objetivo de la práctica
- Entender el problema de **buscar información** en colecciones de texto.
- Comprender por qué se necesita un **índice invertido** en recuperación de información.
- Programar una primera solución manual y luego optimizarla con un índice.
- Evaluar la mejora en tiempos de búsqueda cuando usamos estructuras adecuadas.

## Parte 1: Búsqueda lineal en documentos

### Actividad
1. Se te proporcionará un conjunto de documentos de texto.
2. Escribe una función que:
   - Lea todos los documentos.
   - Busque una palabra ingresada por el usuario.
   - Muestre en qué documentos aparece la palabra.

In [5]:
import os

# Primero verificar si los archivos existen
archivos = ['../data/01_corpus_turismo.txt', '../data/01_corpus_turismo_500.txt']

for archivo in archivos:
    if not os.path.exists(archivo):
        print(f"❌ No se encontró el archivo: {archivo}")
    else:
        print(f"✅ Archivo encontrado: {archivo}")

# Ahora sí correr la búsqueda
def busqueda_lineal(palabra_clave, archivos):
    documentos_con_palabra = []

    for archivo in archivos:
        if os.path.exists(archivo):  # Extra seguridad
            with open(archivo, 'r', encoding='utf-8') as f:
                contenido = f.read()
                if palabra_clave.lower() in contenido.lower():
                    documentos_con_palabra.append(archivo)

    return documentos_con_palabra

palabra = input("🔍 Ingresa la palabra a buscar: ")
resultados = busqueda_lineal(palabra, archivos)

if resultados:
    print(f"La palabra '{palabra}' aparece en los siguientes documentos:")
    for doc in resultados:
        print(f"📄 {doc}")
else:
    print(f"La palabra '{palabra}' no se encontró en ninguno de los documentos.")


✅ Archivo encontrado: ../data/01_corpus_turismo.txt
✅ Archivo encontrado: ../data/01_corpus_turismo_500.txt
La palabra 'Otavalo' aparece en los siguientes documentos:
📄 ../data/01_corpus_turismo.txt
📄 ../data/01_corpus_turismo_500.txt


## Parte 2: Construcción de un índice invertido

### Actividad
1. Escribe un programa que:
   - Recorra todos los documentos.
   - Construya un **índice invertido**, es decir, un diccionario donde:
     - Cada palabra clave apunta a una lista de documentos donde aparece.

2. Escribe una nueva función de búsqueda que:
   - Consulte directamente el índice para encontrar los documentos relevantes.
   - Sea mucho más rápida que la búsqueda lineal.

In [ ]:
import os
import re
from collections import defaultdict

archivos = ['../data/01_corpus_turismo.txt', '../data/01_corpus_turismo_500.txt']

#Indice invertido
def construir_indice_invertido(archivos):
    indice_invertido = defaultdict(set)  # usamos set para evitar documentos repetidos

    for archivo in archivos:
        if os.path.exists(archivo):
            with open(archivo, 'r', encoding='utf-8') as f:
                contenido = f.read()
                # Limpiamos el texto: eliminamos signos de puntuación, etc.
                palabras = re.findall(r'\b\w+\b', contenido.lower())
                for palabra in palabras:
                    indice_invertido[palabra].add(archivo)
    
    return indice_invertido

# Función de búsqueda rápida
def busqueda_por_indice(palabra_clave, indice_invertido):
    palabra_clave = palabra_clave.lower()
    if palabra_clave in indice_invertido:
        return list(indice_invertido[palabra_clave])
    else:
        return []

#Indice
indice = construir_indice_invertido(archivos)

# Consulta
palabra = input("🔎 Ingresa la palabra a buscar (índice invertido): ")
resultados = busqueda_por_indice(palabra, indice)

#Resultados
if resultados:
    print(f"La palabra '{palabra}' aparece en los documentos:")
    for doc in resultados:
        print(f"📄 {doc}")
else:
    print(f"La palabra '{palabra}' no se encontró en ninguno de los documentos.")


La palabra 'Otavalo' aparece en los documentos:
📄 ../data/01_corpus_turismo_500.txt
📄 ../data/01_corpus_turismo.txt


## Parte 3: Evaluación de tiempos de búsqueda
### Actividad

1. Realiza la búsqueda de varias palabras usando:
      -  Corpus pequeño: 16 documentos (turismo en Ecuador).
      -  Corpus grande: 500 documentos (versión ampliada).
2. Mide el tiempo de ejecución:
      -  Para búsqueda lineal.
      -  Para búsqueda usando índice invertido.
      -  Grafica o presenta los resultados en una tabla comparativa.

### Ejemplo de palabras para buscar
- quito
- montañita
- feriado
- playas
- aventura
- galápagos

In [8]:
import os
import re
import time
from collections import defaultdict

archivos = ['../data/01_corpus_turismo.txt', '../data/01_corpus_turismo_500.txt']

#Búsqueda lineal
def busqueda_lineal(palabra_clave, archivos):
    documentos_con_palabra = []
    palabra_clave = palabra_clave.lower()

    for archivo in archivos:
        if os.path.exists(archivo):
            with open(archivo, 'r', encoding='utf-8') as f:
                contenido = f.read()
                if palabra_clave in contenido.lower():
                    documentos_con_palabra.append(archivo)
    return documentos_con_palabra

#Indice invertido ---
def construir_indice_invertido(archivos):
    indice_invertido = defaultdict(set)

    for archivo in archivos:
        if os.path.exists(archivo):
            with open(archivo, 'r', encoding='utf-8') as f:
                contenido = f.read()
                palabras = re.findall(r'\b\w+\b', contenido.lower())
                for palabra in palabras:
                    indice_invertido[palabra].add(archivo)
    return indice_invertido

#Búsqueda por índice invertido ---
def busqueda_por_indice(palabra_clave, indice_invertido):
    palabra_clave = palabra_clave.lower()
    return list(indice_invertido.get(palabra_clave, []))

#Evaluación de tiempos 
palabras_a_buscar = ['quito', 'montañita', 'feriado', 'playas', 'aventura', 'galápagos']

# Construimos el índice invertido antes de medir
indice = construir_indice_invertido(archivos)
resultados_tiempos = []

for palabra in palabras_a_buscar:
    # Tiempo búsqueda lineal
    inicio_lineal = time.perf_counter()
    _ = busqueda_lineal(palabra, archivos)
    fin_lineal = time.perf_counter()
    tiempo_lineal = fin_lineal - inicio_lineal

    # Tiempo búsqueda índice
    inicio_indice = time.perf_counter()
    _ = busqueda_por_indice(palabra, indice)
    fin_indice = time.perf_counter()
    tiempo_indice = fin_indice - inicio_indice

    resultados_tiempos.append((palabra, tiempo_lineal, tiempo_indice))

#Mostrar los resultados
print("\nTabla Comparativa de Tiempos de Búsqueda:\n")
print("{:<12} {:>20} {:>20}".format("Palabra", "Lineal (s)", "Índice (s)"))
print("-" * 60)
for palabra, tiempo_lineal, tiempo_indice in resultados_tiempos:
    print("{:<12} {:>20.6f} {:>20.6f}".format(palabra, tiempo_lineal, tiempo_indice))



Tabla Comparativa de Tiempos de Búsqueda:

Palabra                Lineal (s)           Índice (s)
------------------------------------------------------------
quito                    0.000836             0.000004
montañita                0.000593             0.000003
feriado                  0.000640             0.000003
playas                   0.000641             0.000003
aventura                 0.000557             0.000003
galápagos                0.000559             0.000003


## Parte 4:
### Actividad
1. Modifica el índice para que ignore mayúsculas/minúsculas (por ejemplo, "Playa" y "playa" deben considerarse iguales).
2. Permite consultas de múltiples términos (ejemplo: buscar documentos que contengan "playa" y "turismo").
3. Calcula el _speedup_

In [ ]:
import os
import re
import time
from collections import defaultdict

archivos = ['../data/01_corpus_turismo.txt', '../data/01_corpus_turismo_500.txt']

#Indice invertido (ignorando mayúsculas/minúsculas)
def construir_indice_invertido(archivos):
    indice_invertido = defaultdict(set)

    for archivo in archivos:
        if os.path.exists(archivo):
            with open(archivo, 'r', encoding='utf-8') as f:
                contenido = f.read()
                palabras = re.findall(r'\b\w+\b', contenido.lower())
                for palabra in palabras:
                    indice_invertido[palabra].add(archivo)
    return indice_invertido

#Búsqueda múltiple (acepta varios términos)
def busqueda_multiple_por_indice(terminos, indice_invertido):
    resultados = set()

    for termino in terminos:
        termino = termino.lower()
        if termino in indice_invertido:
            if not resultados:
                resultados = indice_invertido[termino].copy()
            else:
                resultados = resultados.intersection(indice_invertido[termino])

    return list(resultados)

def busqueda_multiple_lineal(terminos, archivos):
    documentos_con_terminos = []
    terminos = [t.lower() for t in terminos]  

    for archivo in archivos:
        if os.path.exists(archivo):
            with open(archivo, 'r', encoding='utf-8') as f:
                contenido = f.read().lower()
                if all(termino in contenido for termino in terminos):
                    documentos_con_terminos.append(archivo)

    return documentos_con_terminos

#Indice invertido
indice = construir_indice_invertido(archivos)

#Consulta de múltiples términos
consulta = input("🔎 Ingresa las palabras que quieres buscar (separadas por espacio): ")
terminos = consulta.strip().split()

#Medir tiempos
inicio_lineal = time.perf_counter()
resultados_lineal = busqueda_multiple_lineal(terminos, archivos)
fin_lineal = time.perf_counter()
tiempo_lineal = fin_lineal - inicio_lineal

inicio_indice = time.perf_counter()
resultados_indice = busqueda_multiple_por_indice(terminos, indice)
fin_indice = time.perf_counter()
tiempo_indice = fin_indice - inicio_indice

#Calcular el speedup
if tiempo_indice > 0:
    speedup = tiempo_lineal / tiempo_indice
else:
    speedup = float('inf')

# 7. Mostrar resultados
print("\nResultados de búsqueda:\n")
print("🔵 Búsqueda Lineal: ", resultados_lineal)
print("🟢 Búsqueda Índice: ", resultados_indice)

print("\n📈 Tiempos:")
print(f"- Búsqueda Lineal: {tiempo_lineal:.6f} segundos")
print(f"- Búsqueda Índice: {tiempo_indice:.6f} segundos")

print(f"\n🚀 Speedup: {speedup:.2f}x más rápido usando índice invertido")



Resultados de búsqueda:

🔵 Búsqueda Lineal:  ['../data/01_corpus_turismo.txt', '../data/01_corpus_turismo_500.txt']
🟢 Búsqueda Índice:  ['../data/01_corpus_turismo_500.txt', '../data/01_corpus_turismo.txt']

📈 Tiempos:
- Búsqueda Lineal: 0.001104 segundos
- Búsqueda Índice: 0.000053 segundos

🚀 Speedup: 20.71x más rápido usando índice invertido
